# Air cargo loading

his task aims to study the optimization process of cargo loading to improve the fuel efficiency, while meeting the constraint of the aircraft center of gravity.

## Example 1: COM

In [2]:
import numpy as np
from scipy.optimize import linprog


def cargo_load_planning_linear1(weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u, cg_impact_4u,
                        max_positions):
    """
    使用整数线性规划方法计算货物装载方案，最小化重心的变化量。

    参数:
        weights (list): 每个货物的质量列表。
        cargo_names (list): 每个货物的名称。
        cargo_types_dict (dict): 货物名称和占用的货位数量。
        positions (list): 可用的货位编号。
        cg_impact (list): 每个位置每kg货物对重心index的影响系数。
        cg_impact_2u (list): 两个位置组合的重心影响系数。
        cg_impact_4u (list): 四个位置组合的重心影响系数。
        max_positions (int): 总货位的数量。

    返回:
        result.x: 最优装载方案矩阵。
    """
    # 将货物类型映射为对应的占用单位数
    cargo_types = [cargo_types_dict[name] for name in cargo_names]

    num_cargos = len(weights)  # 货物数量
    num_positions = len(positions)  # 可用货位数量

    # 决策变量：xij (是否将货物i放置在位置j)
    c = []  # 目标函数系数列表
    for i in range(num_cargos):
        for j in range(num_positions):
            if cargo_types[i] == 1:
                c.append(abs(weights[i] * cg_impact[j]))
            elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                c.append(abs(weights[i] * cg_impact_2u[j // 2]))
            elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                c.append(abs(weights[i] * cg_impact_4u[j // 4]))
            else:
                c.append(0)  # 不适合的索引默认影响为0

    # 决策变量约束：xij只能是0或1 (整型约束由 linprog 近似处理)
    bounds = [(0, 1) for _ in range(num_cargos * num_positions)]

    # 约束1：每个货物只能装载到一个位置
    A_eq = []
    b_eq = []
    for i in range(num_cargos):
        constraint = [0] * (num_cargos * num_positions)
        for j in range(num_positions):
            constraint[i * num_positions + j] = 1
        A_eq.append(constraint)
        b_eq.append(1)

    # 约束2：每个位置只能装载一个货物
    A_ub = []
    b_ub = []
    for j in range(num_positions):  # 遍历所有位置
        constraint = [0] * (num_cargos * num_positions)
        for i in range(num_cargos):  # 遍历所有货物
            constraint[i * num_positions + j] = 1
        A_ub.append(constraint)
        b_ub.append(1)  # 每个位置最多只能分配一个货物

    # 约束3：占用多个位置的货物
    for i, cargo_type in enumerate(cargo_types):
        if cargo_type == 2:  # 两个连续位置组合
            for j in range(0, num_positions - 1, 2):
                constraint = [0] * (num_cargos * num_positions)
                constraint[i * num_positions + j] = 1
                constraint[i * num_positions + j + 1] = 1
                A_ub.append(constraint)
                b_ub.append(1)
        elif cargo_type == 4:  # 上两个、下两个组合
            for j in range(0, num_positions - 3, 4):
                constraint = [0] * (num_cargos * num_positions)
                constraint[i * num_positions + j] = 1
                constraint[i * num_positions + j + 1] = 1
                constraint[i * num_positions + j + 2] = 1
                constraint[i * num_positions + j + 3] = 1
                A_ub.append(constraint)
                b_ub.append(1)

    # 转换为numpy数组
    A_eq = np.array(A_eq)
    b_eq = np.array(b_eq)
    A_ub = np.array(A_ub)
    b_ub = np.array(b_ub)
    c = np.array(c)

    # 求解线性规划问题
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

    if result.success:
        # 解决方案
        solution = result.x.reshape((num_cargos, num_positions))

        # 计算最终重心变化
        cg_change = 0
        for i in range(num_cargos):
            for j in range(num_positions):
                if cargo_types[i] == 1:
                    cg_change += solution[i, j] * weights[i] * cg_impact[j]
                elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                    cg_change += solution[i, j] * weights[i] * cg_impact_2u[j // 2]
                elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                    cg_change += solution[i, j] * weights[i] * cg_impact_4u[j // 4]


        # 输出货物实际分布
        print("货物实际分布:")
        for i in range(num_cargos):
            assigned_positions = []
            for j in range(num_positions):
                if solution[i, j] > 0.5:  # 判断位置是否被分配
                    assigned_positions.append(j)
            print(f"货物 {cargo_names[i]} (占 {cargo_types[i]} 单位): 放置位置 -> {assigned_positions}")

        # 输出1号LD3的摆放位置
        ld3_positions = []
        for j in range(num_positions):
            if solution[0, j] > 0.5:  # LD3 是第一个货物
                ld3_positions.append(j)

        print(f"1号LD3的摆放位置: {ld3_positions}")
        return result, cg_change

    else:
        return None, -1000000


# 示例输入
def main():
    weights = [500, 800, 1200, 300, 700, 1000, 600, 900]  # 每个货物的质量
    cargo_names = ['LD3', 'LD3', 'PLA', 'LD3', 'P6P', 'PLA', 'LD3', 'BULK']  # 货物名称
    cargo_types_dict = {"LD3": 1, "PLA": 2, "P6P": 4, "BULK": 1}  # 货物占位关系
    positions = list(range(44))  # 44个货位编号
    cg_impact = [i * 2 / 43 - 1 for i in range(44)]  # 每kg货物对重心index的影响系数 (单个位置)
    cg_impact_2u = [i * 0.08 for i in range(22)]  # 两个位置组合的影响系数
    cg_impact_4u = [i * 0.05 for i in range(11)]  # 四个位置组合的影响系数
    max_positions = 44  # 总货位数量

    result, cg_change = cargo_load_planning_linear1(weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u,
                                 cg_impact_4u, max_positions)

    if result:
        print("装载方案计算成功！")
    else:
        print("装载方案计算失败！")


if __name__ == "__main__":
    main()

重心的变化量: 30.23
货物实际分布:
货物 LD3 (占 1 单位): 放置位置 -> [23]
货物 LD3 (占 1 单位): 放置位置 -> [21]
货物 PLA (占 2 单位): 放置位置 -> [1]
货物 LD3 (占 1 单位): 放置位置 -> [24]
货物 P6P (占 4 单位): 放置位置 -> [41]
货物 PLA (占 2 单位): 放置位置 -> [29]
货物 LD3 (占 1 单位): 放置位置 -> [20]
货物 BULK (占 1 单位): 放置位置 -> [22]
1号LD3的摆放位置: [23]
装载方案计算成功！


## Example 1: IOM

In [3]:
import numpy as np
from scipy.optimize import linprog


def cargo_load_planning_linear2(weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u, cg_impact_4u,
                        max_positions):
    """
    使用整数线性规划方法计算货物装载方案，最小化重心的变化量。

    参数:
        weights (list): 每个货物的质量列表。
        cargo_names (list): 每个货物的名称。
        cargo_types_dict (dict): 货物名称和占用的货位数量。
        positions (list): 可用的货位编号。
        cg_impact (list): 每个位置每kg货物对重心index的影响系数。
        cg_impact_2u (list): 两个位置组合的重心影响系数。
        cg_impact_4u (list): 四个位置组合的重心影响系数。
        max_positions (int): 总货位的数量。

    返回:
        result.x: 最优装载方案矩阵。
    """
    # 将货物类型映射为对应的占用单位数
    cargo_types = [cargo_types_dict[name] for name in cargo_names]

    num_cargos = len(weights)  # 货物数量
    num_positions = len(positions)  # 可用货位数量

    # 决策变量：xij (是否将货物i放置在位置j)
    c = []  # 目标函数系数列表
    for i in range(num_cargos):
        for j in range(num_positions):
            if cargo_types[i] == 1:
                c.append(abs(weights[i] * cg_impact[j]))
            elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                c.append(abs(weights[i] * cg_impact_2u[j // 2]))
            elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                c.append(abs(weights[i] * cg_impact_4u[j // 4]))
            else:
                c.append(0)  # 不适合的索引默认影响为0

    # 决策变量约束：xij只能是0或1 (整型约束由 linprog 近似处理)
    bounds = [(0, 1) for _ in range(num_cargos * num_positions)]

    # 约束1：每个货物只能装载到一个位置
    A_eq = []
    b_eq = []
    for i in range(num_cargos):
        constraint = [0] * (num_cargos * num_positions)
        for j in range(num_positions):
            constraint[i * num_positions + j] = 1
        A_eq.append(constraint)
        b_eq.append(1)

    # 约束2：每个位置只能装载一个货物
    A_ub = []
    b_ub = []
    for j in range(num_positions):  # 遍历所有位置
        constraint = [0] * (num_cargos * num_positions)
        for i in range(num_cargos):  # 遍历所有货物
            constraint[i * num_positions + j] = 1
        A_ub.append(constraint)
        b_ub.append(1)  # 每个位置最多只能分配一个货物

    # 约束3：占用多个位置的货物
    for i, cargo_type in enumerate(cargo_types):
        if cargo_type == 2:  # 两个连续位置组合
            for j in range(0, num_positions - 1, 2):
                constraint = [0] * (num_cargos * num_positions)
                constraint[i * num_positions + j] = 1
                constraint[i * num_positions + j + 1] = 1
                A_ub.append(constraint)
                b_ub.append(1)
        elif cargo_type == 4:  # 上两个、下两个组合
            for j in range(0, num_positions - 3, 4):
                constraint = [0] * (num_cargos * num_positions)
                constraint[i * num_positions + j] = 1
                constraint[i * num_positions + j + 1] = 1
                constraint[i * num_positions + j + 2] = 1
                constraint[i * num_positions + j + 3] = 1
                A_ub.append(constraint)
                b_ub.append(1)

    # 转换为numpy数组
    A_eq = np.array(A_eq)
    b_eq = np.array(b_eq)
    A_ub = np.array(A_ub)
    b_ub = np.array(b_ub)
    c = np.array(c)

    # 求解线性规划问题
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs-ds')

    if result.success:
        # 解决方案
        solution = result.x.reshape((num_cargos, num_positions))

        # 计算最终重心变化
        cg_change = 0
        for i in range(num_cargos):
            for j in range(num_positions):
                if cargo_types[i] == 1:
                    cg_change += solution[i, j] * weights[i] * cg_impact[j]
                elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                    cg_change += solution[i, j] * weights[i] * cg_impact_2u[j // 2]
                elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                    cg_change += solution[i, j] * weights[i] * cg_impact_4u[j // 4]


        # 输出货物实际分布
        print("货物实际分布:")
        for i in range(num_cargos):
            assigned_positions = []
            for j in range(num_positions):
                if solution[i, j] > 0.5:  # 判断位置是否被分配
                    assigned_positions.append(j)
            print(f"货物 {cargo_names[i]} (占 {cargo_types[i]} 单位): 放置位置 -> {assigned_positions}")

        # 输出1号LD3的摆放位置
        ld3_positions = []
        for j in range(num_positions):
            if solution[0, j] > 0.5:  # LD3 是第一个货物
                ld3_positions.append(j)

        print(f"1号LD3的摆放位置: {ld3_positions}")
        return result, cg_change

    else:
        return None, -1000000


# 示例输入
def main():
    weights = [500, 800, 1200, 300, 700, 1000, 600, 900]  # 每个货物的质量
    cargo_names = ['LD3', 'LD3', 'PLA', 'LD3', 'P6P', 'PLA', 'LD3', 'BULK']  # 货物名称
    cargo_types_dict = {"LD3": 1, "PLA": 2, "P6P": 4, "BULK": 1}  # 货物占位关系
    positions = list(range(44))  # 44个货位编号
    cg_impact = [i * 0.1 for i in range(44)]  # 每kg货物对重心index的影响系数 (单个位置)
    cg_impact_2u = [i * 0.08 for i in range(22)]  # 两个位置组合的影响系数
    cg_impact_4u = [i * 0.05 for i in range(11)]  # 四个位置组合的影响系数
    max_positions = 44  # 总货位数量

    # 归一化 cg_impact 到 [-1, 1]
    cg_impact_min = min(cg_impact)
    cg_impact_max = max(cg_impact)
    cg_impact = [(2 * (x - cg_impact_min) / (cg_impact_max - cg_impact_min)) - 1 for x in cg_impact]

    result, cg_change = cargo_load_planning_linear2(weights, cargo_names, cargo_types_dict, positions, cg_impact,
                                                    cg_impact_2u, cg_impact_4u, max_positions)
    print(f"最小重心变化量: {cg_change:.2f}")


if __name__ == "__main__":
    main()

重心的变化量: 30.23
货物实际分布:
货物 LD3 (占 1 单位): 放置位置 -> [23]
货物 LD3 (占 1 单位): 放置位置 -> [21]
货物 PLA (占 2 单位): 放置位置 -> [1]
货物 LD3 (占 1 单位): 放置位置 -> [24]
货物 P6P (占 4 单位): 放置位置 -> [41]
货物 PLA (占 2 单位): 放置位置 -> [29]
货物 LD3 (占 1 单位): 放置位置 -> [20]
货物 BULK (占 1 单位): 放置位置 -> [22]
1号LD3的摆放位置: [23]
最小重心变化量: 30.23


## Example 3: NL-CPLEX

In [6]:
import numpy as np
from scipy.optimize import minimize


def cargo_load_planning_nonlinear1(weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u, cg_impact_4u,
                        max_positions):
    """
    使用非线性优化方法计算货物装载方案，最小化重心的变化量。

    参数:
        weights (list): 每个货物的质量列表。
        cargo_names (list): 每个货物的名称。
        cargo_types_dict (dict): 货物名称和占用的货位数量。
        positions (list): 可用的货位编号。
        cg_impact (list): 每个位置每kg货物对重心index的影响系数。
        cg_impact_2u (list): 两个位置组合的重心影响系数。
        cg_impact_4u (list): 四个位置组合的重心影响系数。
        max_positions (int): 总货位数量。

    返回:
        result.x: 最优装载方案矩阵。
    """
    # 将货物类型映射为对应的占用单位数
    cargo_types = [cargo_types_dict[name] for name in cargo_names]

    num_cargos = len(weights)  # 货物数量
    num_positions = len(positions)  # 可用货位数量

    # 决策变量：xij (是否将货物i放置在位置j)
    def objective(x):
        cg_change = 0
        idx = 0
        for i in range(num_cargos):
            for j in range(num_positions):
                if cargo_types[i] == 1:
                    cg_change += x[idx] * (weights[i] * cg_impact[j])**2
                elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                    cg_change += x[idx] * (weights[i] * cg_impact_2u[j // 2])**2
                elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                    cg_change += x[idx] * (weights[i] * cg_impact_4u[j // 4])**2
                idx += 1
        return cg_change

    # 约束：每个货物只能装载到一个位置
    cons = []
    for i in range(num_cargos):
        cons.append({
            'type': 'eq',  # 等式约束
            'fun': lambda x, i=i: np.sum(x[i * num_positions:(i + 1) * num_positions]) - 1
        })

    # 约束：每个位置只能装载一个货物
    for j in range(num_positions):
        cons.append({
            'type': 'eq',  # 等式约束
            'fun': lambda x, j=j: np.sum(x[j::num_positions]) - 1
        })

    # 约束：占用多个位置的货物
    for i, cargo_type in enumerate(cargo_types):
        if cargo_type == 2:  # 两个连续位置组合
            for j in range(0, num_positions - 1, 2):
                cons.append({
                    'type': 'eq',
                    'fun': lambda x, i=i, j=j: x[i * num_positions + j] + x[i * num_positions + j + 1] - 1
                })
        elif cargo_type == 4:  # 四个连续位置组合
            for j in range(0, num_positions - 3, 4):
                cons.append({
                    'type': 'eq',
                    'fun': lambda x, i=i, j=j: x[i * num_positions + j] + x[i * num_positions + j + 1] +
                                               x[i * num_positions + j + 2] + x[i * num_positions + j + 3] - 1
                })

    # 设置边界，变量只能是0或1
    bounds = [(0, 1) for _ in range(num_cargos * num_positions)]

    # 初始猜测 (假设每个货物都随机放置在一个位置)
    x0 = np.zeros(num_cargos * num_positions)

    # 使用SLSQP方法进行非线性优化求解
    result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)

    if result.success:
        solution = result.x.reshape((num_cargos, num_positions))

        # 计算最终重心变化
        cg_change = 0
        idx = 0
        for i in range(num_cargos):
            for j in range(num_positions):
                if cargo_types[i] == 1:
                    cg_change += solution[i, j] * (weights[i] * cg_impact[j])**2
                elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                    cg_change += solution[i, j] * (weights[i] * cg_impact_2u[j // 2])**2
                elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                    cg_change += solution[i, j] * (weights[i] * cg_impact_4u[j // 4])**2
                idx += 1

        return result, cg_change
    else:
        return None, -1000000


# 示例输入
def main():
    weights = [500, 800, 1200, 300, 700, 1000, 600, 900]  # 每个货物的质量
    cargo_names = ['LD3', 'LD3', 'PLA', 'LD3', 'P6P', 'PLA', 'LD3', 'BULK']  # 货物名称
    cargo_types_dict = {"LD3": 1, "PLA": 2, "P6P": 4, "BULK": 1}  # 货物占位关系
    positions = list(range(44))  # 44个货位编号
    cg_impact = [i * 0.1 for i in range(44)]  # 每kg货物对重心index的影响系数 (单个位置)
    cg_impact_2u = [i * 0.08 for i in range(22)]  # 两个位置组合的影响系数
    cg_impact_4u = [i * 0.05 for i in range(11)]  # 四个位置组合的影响系数
    max_positions = 44  # 总货位数量

    # 归一化 cg_impact 到 [-1, 1]
    cg_impact_min = min(cg_impact)
    cg_impact_max = max(cg_impact)
    cg_impact = [(2 * (x - cg_impact_min) / (cg_impact_max - cg_impact_min)) - 1 for x in cg_impact]

    result, cg_change = cargo_load_planning_nonlinear1(weights, cargo_names, cargo_types_dict, positions, cg_impact,
                                                    cg_impact_2u, cg_impact_4u, max_positions)



if __name__ == "__main__":
    main()



## Example 4: SDCCLPM

In [ ]:
import numpy as np
from scipy.optimize import linprog


def cargo_load_planning_nonlinear2(weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u, cg_impact_4u,
                        max_positions):
    """
    使用整数线性规划方法计算货物装载方案，最小化重心的变化量。

    参数:
        weights (list): 每个货物的质量列表。
        cargo_names (list): 每个货物的名称。
        cargo_types_dict (dict): 货物名称和占用的货位数量。
        positions (list): 可用的货位编号。
        cg_impact (list): 每个位置每kg货物对重心index的影响系数。
        cg_impact_2u (list): 两个位置组合的重心影响系数。
        cg_impact_4u (list): 四个位置组合的重心影响系数。
        max_positions (int): 总货位数量。

    返回:
        result.x: 最优装载方案矩阵。
    """
    # 将货物类型映射为对应的占用单位数
    cargo_types = [cargo_types_dict[name] for name in cargo_names]

    num_cargos = len(weights)  # 货物数量
    num_positions = len(positions)  # 可用货位数量

    # 决策变量：xij (是否将货物i放置在位置j)
    c = []  # 目标函数系数列表
    for i in range(num_cargos):
        for j in range(num_positions):
            if cargo_types[i] == 1:
                c.append((weights[i] * cg_impact[j])**2)
            elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                c.append((weights[i] * cg_impact_2u[j // 2])**2)
            elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                c.append((weights[i] * cg_impact_4u[j // 4])**2)
            else:
                c.append(0)  # 不适合的索引默认影响为0

    # 决策变量约束：xij只能是0或1 (整型约束由 linprog 近似处理)
    bounds = [(0, 1) for _ in range(num_cargos * num_positions)]

    # 约束1：每个货物只能装载到一个位置
    A_eq = []
    b_eq = []
    for i in range(num_cargos):
        constraint = [0] * (num_cargos * num_positions)
        for j in range(num_positions):
            constraint[i * num_positions + j] = 1
        A_eq.append(constraint)
        b_eq.append(1)

    # 约束2：每个位置只能装载一个货物
    A_ub = []
    b_ub = []
    for j in range(num_positions):  # 遍历所有位置
        constraint = [0] * (num_cargos * num_positions)
        for i in range(num_cargos):  # 遍历所有货物
            constraint[i * num_positions + j] = 1
        A_ub.append(constraint)
        b_ub.append(1)  # 每个位置最多只能分配一个货物

    # 约束3：占用多个位置的货物
    for i, cargo_type in enumerate(cargo_types):
        if cargo_type == 2:  # 两个连续位置组合
            for j in range(0, num_positions - 1, 2):
                constraint = [0] * (num_cargos * num_positions)
                constraint[i * num_positions + j] = 1
                constraint[i * num_positions + j + 1] = 1
                A_ub.append(constraint)
                b_ub.append(1)
        elif cargo_type == 4:  # 上两个、下两个组合
            for j in range(0, num_positions - 3, 4):
                constraint = [0] * (num_cargos * num_positions)
                constraint[i * num_positions + j] = 1
                constraint[i * num_positions + j + 1] = 1
                constraint[i * num_positions + j + 2] = 1
                constraint[i * num_positions + j + 3] = 1
                A_ub.append(constraint)
                b_ub.append(1)

    # 转换为numpy数组
    A_eq = np.array(A_eq)
    b_eq = np.array(b_eq)
    A_ub = np.array(A_ub)
    b_ub = np.array(b_ub)
    c = np.array(c)

    # 求解线性规划问题
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

    if result.success:
        solution = result.x.reshape((num_cargos, num_positions))

        # 计算最终重心变化
        cg_change = 0
        for i in range(num_cargos):
            for j in range(num_positions):
                if cargo_types[i] == 1:
                    cg_change += solution[i, j] * weights[i] * cg_impact[j]
                elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                    cg_change += solution[i, j] * weights[i] * cg_impact_2u[j // 2]
                elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                    cg_change += solution[i, j] * weights[i] * cg_impact_4u[j // 4]

        return result, cg_change
    else:
        return [], -1000000


def main():
    # 示例输入
    weights = [500, 800, 1200, 300, 700, 1000, 600, 900]  # 每个货物的质量
    cargo_names = ['LD3', 'LD3', 'PLA', 'LD3', 'P6P', 'PLA', 'LD3', 'BULK']  # 货物名称
    cargo_types_dict = {"LD3": 1, "PLA": 2, "P6P": 4, "BULK": 1}  # 货物占位关系
    positions = list(range(44))  # 44个货位编号
    cg_impact = [i * 0.1 for i in range(44)]  # 每kg货物对重心index的影响系数 (单个位置)
    cg_impact_2u = [i * 0.08 for i in range(22)]  # 两个位置组合的影响系数
    cg_impact_4u = [i * 0.05 for i in range(11)]  # 四个位置组合的影响系数
    max_positions = 44  # 总货位数量

    # 归一化 cg_impact 到 [-1, 1]
    cg_impact_min = min(cg_impact)
    cg_impact_max = max(cg_impact)
    cg_impact = [(2 * (x - cg_impact_min) / (cg_impact_max - cg_impact_min)) - 1 for x in cg_impact]

    # 调用装载规划函数
    result, cg_change = cargo_load_planning_nonlinear2(weights, cargo_names, cargo_types_dict, positions, cg_impact,
                                                    cg_impact_2u, cg_impact_4u, max_positions)

    if result:
        print(f"最小重心变化量: {cg_change:.2f}")
    else:
        print("没有找到有效的装载方案！")


if __name__ == "__main__":
    main()


## Example 5: MLIP

In [ ]:
import numpy as np
import pulp

def cargo_load_planning_mip(weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u, cg_impact_4u,
                            max_positions):
    """
    使用混合整数规划方法计算货物装载方案，最小化重心的变化量。

    参数:
        weights (list): 每个货物的质量列表。
        cargo_names (list): 每个货物的名称。
        cargo_types_dict (dict): 货物名称和占用的货位数量。
        positions (list): 可用的货位编号。
        cg_impact (list): 每个位置每kg货物对重心index的影响系数。
        cg_impact_2u (list): 两个位置组合的重心影响系数。
        cg_impact_4u (list): 四个位置组合的重心影响系数。
        max_positions (int): 总货位数量。

    返回:
        result (pulp.LpStatus): 求解状态。
        cg_change (float): 最优解的重心变化量。
        solution_matrix (np.ndarray): 最优装载方案矩阵。
    """
    # 将货物类型映射为对应的占用单位数
    cargo_types = [cargo_types_dict[name] for name in cargo_names]

    num_cargos = len(weights)        # 货物数量
    num_positions = len(positions)   # 可用货位数量

    # 创建优化问题实例
    prob = pulp.LpProblem("Cargo_Load_Planning", pulp.LpMinimize)

    # 创建决策变量 x_ij (是否将货物i放置在位置j)
    # 使用字典键 (i,j) 来标识变量
    x = pulp.LpVariable.dicts("x",
                              ((i, j) for i in range(num_cargos) for j in range(num_positions)),
                              cat='Binary')

    # 定义目标函数：最小化重心的变化量
    objective_terms = []
    for i in range(num_cargos):
        for j in range(num_positions):
            if cargo_types[i] == 1:
                impact = abs(weights[i] * cg_impact[j])
            elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                impact = abs(weights[i] * cg_impact_2u[j // 2])
            elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                impact = abs(weights[i] * cg_impact_4u[j // 4])
            else:
                impact = 0
            objective_terms.append(impact * x[i, j])

    prob += pulp.lpSum(objective_terms), "Total_CG_Change"

    # 约束1：每个货物只能装载到一个位置
    for i in range(num_cargos):
        prob += pulp.lpSum([x[i, j] for j in range(num_positions)]) == 1, f"Cargo_{i}_Single_Position"

    # 约束2：每个位置只能装载一个货物
    for j in range(num_positions):
        prob += pulp.lpSum([x[i, j] for i in range(num_cargos)]) <= 1, f"Position_{j}_Single_Cargo"

    # 约束3：占用多个位置的货物
    for i, cargo_type in enumerate(cargo_types):
        if cargo_type == 2:  # 两个连续位置组合
            for j in range(0, num_positions - 1, 2):
                prob += x[i, j] + x[i, j + 1] <= 1, f"Cargo_{i}_Type2_Position_{j}_{j+1}"
        elif cargo_type == 4:  # 四个连续位置组合
            for j in range(0, num_positions - 3, 4):
                prob += x[i, j] + x[i, j + 1] + x[i, j + 2] + x[i, j + 3] <= 1, f"Cargo_{i}_Type4_Position_{j}_{j+3}"

    # 求解问题
    solver = pulp.PULP_CBC_CMD(msg=False)  # 使用默认的CBC求解器，不显示求解过程
    prob.solve(solver)

    # 检查求解状态
    if pulp.LpStatus[prob.status] == 'Optimal':
        # 构建装载方案矩阵
        solution = np.zeros((num_cargos, num_positions))
        for i in range(num_cargos):
            for j in range(num_positions):
                var_value = pulp.value(x[i, j])
                if var_value is not None and var_value > 0.5:
                    solution[i, j] = 1

        # 计算最终重心变化
        cg_change = 0.0
        for i in range(num_cargos):
            for j in range(num_positions):
                if solution[i, j] == 1:
                    if cargo_types[i] == 1:
                        cg_change += weights[i] * cg_impact[j]
                    elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                        cg_change += weights[i] * cg_impact_2u[j // 2]
                    elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                        cg_change += weights[i] * cg_impact_4u[j // 4]

        return solution, cg_change
    else:
        # 若求解失败，则返回空结果和错误标志
        return [], -1000000

def main():
    # 示例输入
    weights = [500, 800, 1200, 300, 700, 1000, 600, 900]  # 每个货物的质量
    cargo_names = ['LD3', 'LD3', 'PLA', 'LD3', 'P6P', 'PLA', 'LD3', 'BULK']  # 货物名称
    cargo_types_dict = {"LD3": 1, "PLA": 2, "P6P": 4, "BULK": 1}  # 货物占位关系
    positions = list(range(44))  # 44个货位编号
    cg_impact = [i * 0.1 for i in range(44)]  # 每kg货物对重心index的影响系数 (单个位置)
    cg_impact_2u = [i * 0.08 for i in range(22)]  # 两个位置组合的影响系数
    cg_impact_4u = [i * 0.05 for i in range(11)]  # 四个位置组合的影响系数
    max_positions = 44  # 总货位数量

    # 归一化 cg_impact 到 [-1, 1]
    cg_impact_min = min(cg_impact)
    cg_impact_max = max(cg_impact)
    cg_impact = [(2 * (x - cg_impact_min) / (cg_impact_max - cg_impact_min)) - 1 for x in cg_impact]

    # 调用装载规划函数
    solution, cg_change = cargo_load_planning_mip(
        weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u, cg_impact_4u, max_positions
    )

    if solution:
        print("成功找到最优装载方案！")
        print("装载方案矩阵:")
        print(solution)

        print(f"重心的变化量: {cg_change:.2f}")

        # 输出实际分布
        for i in range(len(weights)):
            assigned_positions = []
            for j in range(len(positions)):
                if solution[i, j] > 0.5:  # 判断位置是否被分配
                    assigned_positions.append(j)
            print(f"货物 {cargo_names[i]} (占 {cargo_types_dict[cargo_names[i]]} 单位): 放置位置 -> {assigned_positions}")
    else:
        print("未能找到可行解。")
        print(f"求解状态: {pulp.LpStatus[solution]}")

if __name__ == "__main__":
    main()


## Example 6: MLIP-WBP

In [ ]:
import numpy as np
import pulp

def cargo_load_planning_mip(weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u, cg_impact_4u,
                            max_positions):
    """
    使用混合整数规划方法计算货物装载方案，最小化重心的变化量。

    参数:
        weights (list): 每个货物的质量列表。
        cargo_names (list): 每个货物的名称。
        cargo_types_dict (dict): 货物名称和占用的货位数量。
        positions (list): 可用的货位编号。
        cg_impact (list): 每个位置每kg货物对重心index的影响系数。
        cg_impact_2u (list): 两个位置组合的重心影响系数。
        cg_impact_4u (list): 四个位置组合的重心影响系数。
        max_positions (int): 总货位数量。

    返回:
        result (pulp.LpStatus): 求解状态。
        cg_change (float): 最优解的重心变化量。
        solution_matrix (np.ndarray): 最优装载方案矩阵。
    """
    # 将货物类型映射为对应的占用单位数
    cargo_types = [cargo_types_dict[name] for name in cargo_names]

    num_cargos = len(weights)        # 货物数量
    num_positions = len(positions)   # 可用货位数量

    # 创建优化问题实例
    prob = pulp.LpProblem("Cargo_Load_Planning", pulp.LpMinimize)

    # 创建决策变量 x_ij (是否将货物i放置在位置j)
    # 使用字典键 (i,j) 来标识变量
    x = pulp.LpVariable.dicts("x",
                              ((i, j) for i in range(num_cargos) for j in range(num_positions)),
                              cat='Binary')

    # 定义目标函数：最小化重心的变化量
    objective_terms = []
    for i in range(num_cargos):
        for j in range(num_positions):
            if cargo_types[i] == 1:
                impact = abs(weights[i] * cg_impact[j])
            elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                impact = abs(weights[i] * cg_impact_2u[j // 2])
            elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                impact = abs(weights[i] * cg_impact_4u[j // 4])
            else:
                impact = 0
            objective_terms.append(impact * x[i, j])

    prob += pulp.lpSum(objective_terms), "Total_CG_Change"

    # 约束1：每个货物只能装载到一个位置
    for i in range(num_cargos):
        prob += pulp.lpSum([x[i, j] for j in range(num_positions)]) == 1, f"Cargo_{i}_Single_Position"

    # 约束2：每个位置只能装载一个货物
    for j in range(num_positions):
        prob += pulp.lpSum([x[i, j] for i in range(num_cargos)]) <= 1, f"Position_{j}_Single_Cargo"

    # 约束3：占用多个位置的货物
    for i, cargo_type in enumerate(cargo_types):
        if cargo_type == 2:  # 两个连续位置组合
            for j in range(0, num_positions - 1, 2):
                prob += x[i, j] + x[i, j + 1] <= 1, f"Cargo_{i}_Type2_Position_{j}_{j+1}"
        elif cargo_type == 4:  # 四个连续位置组合
            for j in range(0, num_positions - 3, 4):
                prob += x[i, j] + x[i, j + 1] + x[i, j + 2] + x[i, j + 3] <= 1, f"Cargo_{i}_Type4_Position_{j}_{j+3}"

    # 求解问题
    solver = pulp.GLPK_CMD(msg=False)  # 使用GLPK求解器，不显示求解过程
    prob.solve(solver)

    # 检查求解状态
    if pulp.LpStatus[prob.status] == 'Optimal':
        # 构建装载方案矩阵
        solution = np.zeros((num_cargos, num_positions))
        for i in range(num_cargos):
            for j in range(num_positions):
                var_value = pulp.value(x[i, j])
                if var_value is not None and var_value > 0.5:
                    solution[i, j] = 1

        # 计算最终重心变化
        cg_change = 0.0
        for i in range(num_cargos):
            for j in range(num_positions):
                if solution[i, j] == 1:
                    if cargo_types[i] == 1:
                        cg_change += weights[i] * cg_impact[j]
                    elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                        cg_change += weights[i] * cg_impact_2u[j // 2]
                    elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                        cg_change += weights[i] * cg_impact_4u[j // 4]

        return pulp.LpStatus[prob.status], cg_change, solution
    else:
        # 若求解失败，则返回空结果和错误标志
        return pulp.LpStatus[prob.status], -1000000, None

# 示例输入和调用
def main():
    weights = [500, 800, 1200, 300, 700, 1000, 600, 900]  # 每个货物的质量
    cargo_names = ['LD3', 'LD3', 'PLA', 'LD3', 'P6P', 'PLA', 'LD3', 'BULK']  # 货物名称
    cargo_types_dict = {"LD3": 1, "PLA": 2, "P6P": 4, "BULK": 1}  # 货物占位关系
    positions = list(range(44))  # 44个货位编号
    cg_impact = [i * 0.1 for i in range(44)]  # 每kg货物对重心index的影响系数 (单个位置)
    cg_impact_2u = [i * 0.08 for i in range(22)]  # 两个位置组合的影响系数
    cg_impact_4u = [i * 0.05 for i in range(11)]  # 四个位置组合的影响系数
    max_positions = 44  # 总货位数量

    # 归一化 cg_impact 到 [-1, 1]
    cg_impact_min = min(cg_impact)
    cg_impact_max = max(cg_impact)
    cg_impact = [(2 * (x - cg_impact_min) / (cg_impact_max - cg_impact_min)) - 1 for x in cg_impact]

    # 调用装载规划函数
    status, cg_change, solution = cargo_load_planning_mip(
        weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u, cg_impact_4u, max_positions
    )

    if status == 'Optimal':
        print("成功找到最优装载方案！")
        print("装载方案矩阵:")
        print(solution)

        print(f"重心的变化量: {cg_change:.2f}")

        # 输出实际分布
        for i in range(len(weights)):
            assigned_positions = []
            for j in range(len(positions)):
                if solution[i, j] > 0.5:  # 判断位置是否被分配
                    assigned_positions.append(j)
            print(f"货物 {cargo_names[i]} (占 {cargo_types_dict[cargo_names[i]]} 单位): 放置位置 -> {assigned_positions}")
    else:
        print("未能找到可行解。")
        print(f"求解状态: {status}")

if __name__ == "__main__":
    main()


## Example 7: MLIP-SCLPDD

In [ ]:
import numpy as np
from scipy.optimize import linprog


def normalize(cg_impact):
    min_val = min(cg_impact)
    max_val = max(cg_impact)
    return [(2 * (x - min_val) / (max_val - min_val)) - 1 for x in cg_impact]  # 转换到[-1, 1]范围


def cargo_load_planning_linear3(weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u, cg_impact_4u,
                        max_positions):
    """
    使用整数线性规划方法计算货物装载方案，最小化重心的变化量。

    参数:
        weights (list): 每个货物的质量列表。
        cargo_names (list): 每个货物的名称。
        cargo_types_dict (dict): 货物名称和占用的货位数量。
        positions (list): 可用的货位编号。
        cg_impact (list): 每个位置每kg货物对重心index的影响系数。
        cg_impact_2u (list): 两个位置组合的重心影响系数。
        cg_impact_4u (list): 四个位置组合的重心影响系数。
        max_positions (int): 总货位的数量。

    返回:
        result.x: 最优装载方案矩阵。
    """
    # 将货物类型映射为对应的占用单位数
    cargo_types = [cargo_types_dict[name] for name in cargo_names]

    # 归一化重心影响系数到[-1, 1]
    cg_impact = normalize(cg_impact)
    cg_impact_2u = normalize(cg_impact_2u)
    cg_impact_4u = normalize(cg_impact_4u)

    num_cargos = len(weights)  # 货物数量
    num_positions = len(positions)  # 可用货位数量

    # 决策变量：xij (是否将货物i放置在位置j)
    c = []  # 目标函数系数列表
    for i in range(num_cargos):
        for j in range(num_positions):
            if cargo_types[i] == 1:
                c.append(abs(weights[i] * cg_impact[j]))
            elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                c.append(abs(weights[i] * cg_impact_2u[j // 2]))
            elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                c.append(abs(weights[i] * cg_impact_4u[j // 4]))
            else:
                c.append(0)  # 不适合的索引默认影响为0

    # 决策变量约束：xij只能是0或1 (整型约束由 linprog 近似处理)
    bounds = [(0, 1) for _ in range(num_cargos * num_positions)]

    # 约束1：每个货物只能装载到一个位置
    A_eq = []
    b_eq = []
    for i in range(num_cargos):
        constraint = [0] * (num_cargos * num_positions)
        for j in range(num_positions):
            constraint[i * num_positions + j] = 1
        A_eq.append(constraint)
        b_eq.append(1)

    # 约束2：每个位置只能装载一个货物
    A_ub = []
    b_ub = []
    for j in range(num_positions):  # 遍历所有位置
        constraint = [0] * (num_cargos * num_positions)
        for i in range(num_cargos):  # 遍历所有货物
            constraint[i * num_positions + j] = 1
        A_ub.append(constraint)
        b_ub.append(1)  # 每个位置最多只能分配一个货物

    # 约束3：占用多个位置的货物
    for i, cargo_type in enumerate(cargo_types):
        if cargo_type == 2:  # 两个连续位置组合
            for j in range(0, num_positions - 1, 2):
                constraint = [0] * (num_cargos * num_positions)
                constraint[i * num_positions + j] = 1
                constraint[i * num_positions + j + 1] = 1
                A_ub.append(constraint)
                b_ub.append(1)
        elif cargo_type == 4:  # 上两个、下两个组合
            for j in range(0, num_positions - 3, 4):
                constraint = [0] * (num_cargos * num_positions)
                constraint[i * num_positions + j] = 1
                constraint[i * num_positions + j + 1] = 1
                constraint[i * num_positions + j + 2] = 1
                constraint[i * num_positions + j + 3] = 1
                A_ub.append(constraint)
                b_ub.append(1)

    # 转换为numpy数组
    A_eq = np.array(A_eq)
    b_eq = np.array(b_eq)
    A_ub = np.array(A_ub)
    b_ub = np.array(b_ub)
    c = np.array(c)

    # 求解线性规划问题
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs-ipm')

    if result.success:
        # 打印成功找到最优装载方案
        solution = result.x.reshape((num_cargos, num_positions))

        # 计算最终重心变化
        cg_change = 0
        for i in range(num_cargos):
            for j in range(num_positions):
                if cargo_types[i] == 1:
                    cg_change += solution[i, j] * weights[i] * cg_impact[j]
                elif cargo_types[i] == 2 and j % 2 == 0 and j < len(cg_impact_2u) * 2:
                    cg_change += solution[i, j] * weights[i] * cg_impact_2u[j // 2]
                elif cargo_types[i] == 4 and j % 4 == 0 and j < len(cg_impact_4u) * 4:
                    cg_change += solution[i, j] * weights[i] * cg_impact_4u[j // 4]

        return result, cg_change, solution
    else:
        result = []
        return result, -1000000, None


def main():
    # 示例输入
    weights = [500, 800, 1200, 300, 700, 1000, 600, 900]  # 每个货物的质量
    cargo_names = ['LD3', 'LD3', 'PLA', 'LD3', 'P6P', 'PLA', 'LD3', 'BULK']  # 货物名称
    cargo_types_dict = {"LD3": 1, "PLA": 2, "P6P": 4, "BULK": 1}  # 货物占位关系
    positions = list(range(44))  # 44个货位编号
    cg_impact = [i * 0.1 for i in range(44)]  # 每kg货物对重心index的影响系数 (单个位置)
    cg_impact_2u = [i * 0.08 for i in range(22)]  # 两个位置组合的影响系数
    cg_impact_4u = [i * 0.05 for i in range(11)]  # 四个位置组合的影响系数
    max_positions = 44  # 总货位数量

    # 调用装载规划函数
    result, cg_change, solution = cargo_load_planning_linear3(weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u,
                                                               cg_impact_4u, max_positions)

    # 打印结果
    if result.success:
        print("成功找到最优装载方案！")
        print(f"重心变化量: {cg_change:.2f}")

        # 输出装载方案
        print("\n装载方案矩阵:")
        print(solution)

        # 输出每个货物的实际分布位置
        print("\n货物实际分布:")
        for i in range(len(weights)):
            assigned_positions = []
            for j in range(len(positions)):
                if solution[i, j] > 0.5:  # 判断位置是否被分配
                    assigned_positions.append(j)
            print(f"货物 {cargo_names[i]} (占 {cargo_types_dict[cargo_names[i]]} 单位): 放置位置 -> {assigned_positions}")
    else:
        print("未能找到最优装载方案。")


if __name__ == "__main__":
    main()


## Example 8: HGA

In [ ]:
import numpy as np
import random
from deap import base, creator, tools, algorithms


def cargo_load_planning_genetic(weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u, cg_impact_4u,
                                max_positions, population_size=100, generations=100, crossover_prob=0.7, mutation_prob=0.2):
    """
    使用改进版遗传算法计算货物装载方案，最小化重心的变化量。

    参数:
        weights (list): 每个货物的质量列表。
        cargo_names (list): 每个货物的名称。
        cargo_types_dict (dict): 货物名称和占用的货位数量。
        positions (list): 可用的货位编号。
        cg_impact (list): 每个位置每kg货物对重心index的影响系数。
        cg_impact_2u (list): 两个位置组合的重心影响系数。
        cg_impact_4u (list): 四个位置组合的重心影响系数。
        max_positions (int): 总货位的数量。
        population_size (int): 遗传算法的种群大小。
        generations (int): 遗传算法的代数。
        crossover_prob (float): 交叉操作的概率。
        mutation_prob (float): 变异操作的概率。

    返回:
        best_solution (np.array): 最优装载方案矩阵。
        best_cg_change (float): 最优方案的重心变化量。
    """
    try:
        # 将货物类型映射为对应的占用单位数
        cargo_types = [cargo_types_dict[name] for name in cargo_names]

        num_cargos = len(weights)       # 货物数量
        num_positions = len(positions)  # 可用货位数量

        # 定义适应度函数（最小化重心变化量）
        if not hasattr(creator, "FitnessMin"):
            creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # 目标是最小化
        if not hasattr(creator, "Individual"):
            creator.create("Individual", list, fitness=creator.FitnessMin)

        toolbox = base.Toolbox()

        # 个体初始化函数
        def init_individual():
            individual = []
            occupied = [False] * num_positions
            for cargo_type in cargo_types:
                if cargo_type == 1:
                    valid_positions = [j for j in range(num_positions) if not occupied[j]]
                elif cargo_type == 2:
                    valid_positions = [j for j in range(0, num_positions - 1, 2) if not any(occupied[j + k] for k in range(cargo_type))]
                elif cargo_type == 4:
                    valid_positions = [j for j in range(0, num_positions - 3, 4) if not any(occupied[j + k] for k in range(cargo_type))]
                else:
                    valid_positions = []

                if not valid_positions:
                    # 如果没有有效位置，随机选择一个符合类型对齐的起始位置
                    if cargo_type == 1:
                        start_pos = random.randint(0, num_positions - 1)
                    elif cargo_type == 2:
                        choices = [j for j in range(0, num_positions - 1, 2)]
                        if choices:
                            start_pos = random.choice(choices)
                        else:
                            start_pos = 0  # 默认位置
                    elif cargo_type == 4:
                        choices = [j for j in range(0, num_positions - 3, 4)]
                        if choices:
                            start_pos = random.choice(choices)
                        else:
                            start_pos = 0  # 默认位置
                    else:
                        start_pos = 0  # 默认位置
                else:
                    start_pos = random.choice(valid_positions)

                individual.append(start_pos)

                # 标记占用的位置
                for k in range(cargo_type):
                    pos = start_pos + k
                    if pos < num_positions:
                        occupied[pos] = True

            return creator.Individual(individual)

        toolbox.register("individual", init_individual)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # 适应度评估函数
        def evaluate(individual):
            # 检查重叠和边界
            occupied = [False] * num_positions
            penalty = 0
            cg_change = 0.0

            for i, start_pos in enumerate(individual):
                cargo_type = cargo_types[i]
                weight = weights[i]

                # 检查边界
                if start_pos < 0 or start_pos + cargo_type > num_positions:
                    penalty += 10000 # 超出边界的严重惩罚
                    continue

                # 检查重叠
                overlap = False
                for k in range(cargo_type):
                    pos = start_pos + k
                    if occupied[pos]:
                        penalty += 10000  # 重叠的严重惩罚
                        overlap = True
                        break
                    occupied[pos] = True
                if overlap:
                    continue

                # 计算重心变化量
                if cargo_type == 1:
                    cg_change += abs(weight * cg_impact[start_pos])
                elif cargo_type == 2:
                    if start_pos % 2 == 0 and (start_pos // 2) < len(cg_impact_2u):
                        cg_change += abs(weight * cg_impact_2u[start_pos // 2])
                    else:
                        penalty += 10000  # 不对齐的严重惩罚
                elif cargo_type == 4:
                    if start_pos % 4 == 0 and (start_pos // 4) < len(cg_impact_4u):
                        cg_change += abs(weight * cg_impact_4u[start_pos // 4])
                    else:
                        penalty += 10000  # 不对齐的严重惩罚

            return (cg_change + penalty,)

        toolbox.register("evaluate", evaluate)
        toolbox.register("mate", tools.cxOnePoint)  # 改为单点交叉
        toolbox.register("mutate", tools.mutShuffleIndexes, indpb=mutation_prob)  # 使用交换变异
        toolbox.register("select", tools.selRoulette)  # 轮盘赌选择

        # 初始化种群
        population = toolbox.population(n=population_size)

        # 运行遗传算法
        try:
            algorithms.eaSimple(population, toolbox, cxpb=crossover_prob, mutpb=1.0, ngen=generations,
                                verbose=False)
        except ValueError as e:
            print(f"遗传算法运行时出错: {e}")
            return [], -1000000  # 返回空列表和一个负的重心变化量作为错误标志

        # 选择最优个体
        try:
            best_individual = tools.selBest(population, 1)[0]
            best_cg_change = evaluate(best_individual)[0]
        except IndexError as e:
            print(f"选择最优个体时出错: {e}")
            return [], -1000000  # 返回空列表和一个负的重心变化量作为错误标志

        # 构建装载方案矩阵
        solution = np.zeros((num_cargos, num_positions))
        for i, start_pos in enumerate(best_individual):
            cargo_type = cargo_types[i]
            for k in range(cargo_type):
                pos = start_pos + k
                if pos < num_positions:
                    solution[i, pos] = 1

        return solution, best_cg_change
    except Exception as e:
        print(f"发生错误: {e}")
        return [], -1000000


def main():
    # 示例输入
    weights = [500, 800, 1200, 300, 700, 1000, 600, 900]  # 每个货物的质量
    cargo_names = ['LD3', 'LD3', 'PLA', 'LD3', 'P6P', 'PLA', 'LD3', 'BULK']  # 货物名称
    cargo_types_dict = {"LD3": 1, "PLA": 2, "P6P": 4, "BULK": 1}  # 货物占位关系
    positions = list(range(44))  # 44个货位编号
    cg_impact = [i * 0.1 for i in range(44)]  # 每kg货物对重心index的影响系数 (单个位置)
    cg_impact_2u = [i * 0.08 for i in range(22)]  # 两个位置组合的影响系数
    cg_impact_4u = [i * 0.05 for i in range(11)]  # 四个位置组合的影响系数
    max_positions = 44  # 总货位数量

    # 调用遗传算法进行货物装载优化
    best_solution, best_cg_change = cargo_load_planning_genetic(weights, cargo_names, cargo_types_dict, positions,
                                                                cg_impact, cg_impact_2u, cg_impact_4u, max_positions)

    if best_solution:
        print("最优装载方案矩阵：")
        print(best_solution)
        print(f"最优装载方案的重心变化量：{best_cg_change:.2f}")
    else:
        print("优化失败，请检查输入或算法设置。")

if __name__ == "__main__":
    main()

## Example 9: GA-normal

In [ ]:
import numpy as np
import random
from deap import base, creator, tools, algorithms


def cargo_load_planning_genetic(weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u, cg_impact_4u,
                                max_positions, population_size=100, generations=100, crossover_prob=0.7, mutation_prob=0.2):
    """
    使用遗传算法计算货物装载方案，最小化重心的变化量。

    参数:
        weights (list): 每个货物的质量列表。
        cargo_names (list): 每个货物的名称。
        cargo_types_dict (dict): 货物名称和占用的货位数量。
        positions (list): 可用的货位编号。
        cg_impact (list): 每个位置每kg货物对重心index的影响系数。
        cg_impact_2u (list): 两个位置组合的重心影响系数。
        cg_impact_4u (list): 四个位置组合的重心影响系数。
        max_positions (int): 总货位的数量。
        population_size (int): 遗传算法的种群大小。
        generations (int): 遗传算法的代数。
        crossover_prob (float): 交叉操作的概率。
        mutation_prob (float): 变异操作的概率。

    返回:
        best_solution (np.array): 最优装载方案矩阵。
        best_cg_change (float): 最优方案的重心变化量。
    """
    try:
        # 将货物类型映射为对应的占用单位数
        cargo_types = [cargo_types_dict[name] for name in cargo_names]

        num_cargos = len(weights)       # 货物数量
        num_positions = len(positions)  # 可用货位数量

        # 定义适应度函数（最小化重心变化量）
        if not hasattr(creator, "FitnessMin"):
            creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        if not hasattr(creator, "Individual"):
            creator.create("Individual", list, fitness=creator.FitnessMin)

        toolbox = base.Toolbox()

        # 个体初始化函数，确保货物类型为2和4时起始位置对齐
        def init_individual():
            individual = []
            occupied = [False] * num_positions
            for cargo_type in cargo_types:
                if cargo_type == 1:
                    valid_positions = [j for j in range(num_positions) if not occupied[j]]
                elif cargo_type == 2:
                    valid_positions = [j for j in range(0, num_positions - 1, 2) if not any(occupied[j + k] for k in range(cargo_type))]
                elif cargo_type == 4:
                    valid_positions = [j for j in range(0, num_positions - 3, 4) if not any(occupied[j + k] for k in range(cargo_type))]
                else:
                    valid_positions = []

                if not valid_positions:
                    # 如果没有有效位置，随机选择一个符合类型对齐的起始位置
                    if cargo_type == 1:
                        start_pos = random.randint(0, num_positions - 1)
                    elif cargo_type == 2:
                        choices = [j for j in range(0, num_positions - 1, 2)]
                        if choices:
                            start_pos = random.choice(choices)
                        else:
                            start_pos = 0  # 默认位置
                    elif cargo_type == 4:
                        choices = [j for j in range(0, num_positions - 3, 4)]
                        if choices:
                            start_pos = random.choice(choices)
                        else:
                            start_pos = 0  # 默认位置
                    else:
                        start_pos = 0  # 默认位置
                else:
                    start_pos = random.choice(valid_positions)

                individual.append(start_pos)

                # 标记占用的位置
                for k in range(cargo_type):
                    pos = start_pos + k
                    if pos < num_positions:
                        occupied[pos] = True

            return creator.Individual(individual)

        toolbox.register("individual", init_individual)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # 适应度评估函数
        def evaluate(individual):
            # 检查重叠和边界
            occupied = [False] * num_positions
            penalty = 0
            cg_change = 0.0

            for i, start_pos in enumerate(individual):
                cargo_type = cargo_types[i]
                weight = weights[i]

                # 检查边界
                if start_pos < 0 or start_pos + cargo_type > num_positions:
                    penalty += 10000 # 超出边界的严重惩罚
                    continue

                # 检查重叠
                overlap = False
                for k in range(cargo_type):
                    pos = start_pos + k
                    if occupied[pos]:
                        penalty += 10000  # 重叠的严重惩罚
                        overlap = True
                        break
                    occupied[pos] = True
                if overlap:
                    continue

                # 计算重心变化量
                if cargo_type == 1:
                    cg_change += abs(weight * cg_impact[start_pos])
                elif cargo_type == 2:
                    if start_pos % 2 == 0 and (start_pos // 2) < len(cg_impact_2u):
                        cg_change += abs(weight * cg_impact_2u[start_pos // 2])
                    else:
                        penalty += 10000  # 不对齐的严重惩罚
                elif cargo_type == 4:
                    if start_pos % 4 == 0 and (start_pos // 4) < len(cg_impact_4u):
                        cg_change += abs(weight * cg_impact_4u[start_pos // 4])
                    else:
                        penalty += 10000  # 不对齐的严重惩罚

            return (cg_change + penalty,)

        toolbox.register("evaluate", evaluate)
        toolbox.register("mate", tools.cxTwoPoint)

        # 自定义变异函数，确保变异后的起始位置对齐
        def custom_mutate(individual, indpb):
            for i in range(len(individual)):
                if random.random() < indpb:
                    cargo_type = cargo_types[i]
                    try:
                        if cargo_type == 1:
                            new_pos = random.randint(0, num_positions - 1)
                        elif cargo_type == 2:
                            choices = [j for j in range(0, num_positions - 1, 2)]
                            if choices:
                                new_pos = random.choice(choices)
                            else:
                                new_pos = individual[i]  # 如果没有可选位置，保持不变
                        elif cargo_type == 4:
                            choices = [j for j in range(0, num_positions - 3, 4)]
                            if choices:
                                new_pos = random.choice(choices)
                            else:
                                new_pos = individual[i]  # 如果没有可选位置，保持不变
                        else:
                            new_pos = individual[i]  # 保持不变
                        individual[i] = new_pos
                    except ValueError:
                        # 捕获可能的 ValueError 并直接返回当前个体
                        return individual,
            return (individual,)

        toolbox.register("mutate", custom_mutate, indpb=mutation_prob)
        toolbox.register("select", tools.selTournament, tournsize=3)

        # 初始化种群
        population = toolbox.population(n=population_size)

        # 运行遗传算法
        try:
            algorithms.eaSimple(population, toolbox, cxpb=crossover_prob, mutpb=1.0, ngen=generations,
                                verbose=False)
        except ValueError as e:
            print(f"遗传算法运行时出错: {e}")
            return [], -1000000  # 返回空列表和一个负的重心变化量作为错误标志

        # 选择最优个体
        try:
            best_individual = tools.selBest(population, 1)[0]
            best_cg_change = evaluate(best_individual)[0]
        except IndexError as e:
            print(f"选择最优个体时出错: {e}")
            return [], -1000000  # 返回空列表和一个负的重心变化量作为错误标志

        # 构建装载方案矩阵
        solution = np.zeros((num_cargos, num_positions))
        for i, start_pos in enumerate(best_individual):
            cargo_type = cargo_types[i]
            for k in range(cargo_type):
                pos = start_pos + k
                if pos < num_positions:
                    solution[i, pos] = 1

        return solution, best_cg_change
    except:
        return [],-1000000


# 示例输入和调用
def main():
    weights = [500, 800, 1200, 300, 700, 1000, 600, 900]  # 每个货物的质量
    cargo_names = ['LD3', 'LD3', 'PLA', 'LD3', 'P6P', 'PLA', 'LD3', 'BULK']  # 货物名称
    cargo_types_dict = {"LD3": 1, "PLA": 2, "P6P": 4, "BULK": 1}  # 货物占位关系
    positions = list(range(44))  # 44个货位编号
    cg_impact = [i * 0.1 for i in range(44)]  # 每kg货物对重心index的影响系数 (单个位置)
    cg_impact_2u = [i * 0.08 for i in range(22)]  # 两个位置组合的影响系数
    cg_impact_4u = [i * 0.05 for i in range(11)]  # 四个位置组合的影响系数
    max_positions = 44  # 总货位数量

    solution, cg_change = cargo_load_planning_genetic(
        weights, cargo_names, cargo_types_dict, positions,
        cg_impact, cg_impact_2u, cg_impact_4u, max_positions,
        population_size=200, generations=200, crossover_prob=0.8, mutation_prob=0.2
    )

    if solution is not None and len(solution) > 0:
        print("成功找到最优装载方案！")
        print("装载方案矩阵:")
        print(solution)
        print(f"重心的变化量: {cg_change:.2f}")

        # 输出实际分布
        for i in range(len(weights)):
            assigned_positions = []
            for j in range(len(positions)):
                if solution[i, j] > 0.5:  # 判断位置是否被分配
                    assigned_positions.append(j)
            print(f"货物 {cargo_names[i]} (占 {cargo_types_dict[cargo_names[i]]} 单位): 放置位置 -> {assigned_positions}")
    else:
        print("未找到可行的装载方案。")


if __name__ == "__main__":
    main()


## Example 10: DMOPSO

In [ ]:
import numpy as np
import pyswarms as ps

def cargo_load_planning_pso_v2(weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u, cg_impact_4u,
                               max_positions, options=None, swarmsize=100, maxiter=100):
    """
    使用二进制粒子群优化方法计算货物装载方案，最小化重心的变化量。

    参数:
        weights (list): 每个货物的质量列表。
        cargo_names (list): 每个货物的名称。
        cargo_types_dict (dict): 货物名称和占用的货位数量。
        positions (list): 可用的货位编号。
        cg_impact (list): 每个位置每kg货物对重心index的影响系数。
        cg_impact_2u (list): 两个位置组合的重心影响系数。
        cg_impact_4u (list): 四个位置组合的重心影响系数。
        max_positions (int): 总货位的数量。
        options (dict, optional): PSO算法的配置选项。
        swarmsize (int, optional): 粒子群大小。
        maxiter (int, optional): 最大迭代次数。

    返回:
        best_solution (np.array): 最优装载方案矩阵。
        best_cg_change (float): 最优方案的重心变化量。
    """
    # 将货物类型映射为对应的占用单位数
    cargo_types = [cargo_types_dict[name] for name in cargo_names]

    num_cargos = len(weights)  # 货物数量
    num_positions = len(positions)  # 可用货位数量
    dimension = num_cargos * max_positions  # 每个粒子的维度：货物数量 × 可用货位数量

    # 如果未提供options，使用默认配置
    if options is None:
        options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

    # 定义适应度评估函数
    def fitness_function(x):
        """
        计算每个粒子的适应度值。

        参数:
            x (numpy.ndarray): 粒子的位置数组，形状为 (n_particles, dimension)。

        返回:
            numpy.ndarray: 每个粒子的适应度值。
        """
        fitness = np.zeros(x.shape[0])

        for idx, particle in enumerate(x):
            # 将连续位置映射为离散起始位置
            start_positions = []
            penalty = 0
            cg_change = 0.0
            occupied = np.zeros(num_positions, dtype=int)

            for i in range(num_cargos):
                cargo_type = cargo_types[i]
                pos_continuous = particle[i * max_positions:(i + 1) * max_positions]

                # 根据粒子位置值选择最佳货位
                start_pos = np.argmax(pos_continuous)

                # 检查边界
                if start_pos < 0 or start_pos + cargo_type > num_positions:
                    penalty += 1000
                    continue

                # 检查对齐
                if cargo_type == 2 and start_pos % 2 != 0:
                    penalty += 1000
                if cargo_type == 4 and start_pos % 4 != 0:
                    penalty += 1000

                # 检查重叠
                if np.any(occupied[start_pos:start_pos + cargo_type]):
                    penalty += 1000
                else:
                    occupied[start_pos:start_pos + cargo_type] = 1

                start_positions.append(start_pos)

                # 计算重心变化量
                if cargo_type == 1:
                    cg_change += weights[i] * cg_impact[start_pos]
                elif cargo_type == 2:
                    cg_change += weights[i] * cg_impact_2u[start_pos // 2]
                elif cargo_type == 4:
                    cg_change += weights[i] * cg_impact_4u[start_pos // 4]

            fitness[idx] = cg_change + penalty

        return fitness

    # 设置PSO的边界
    # 对于每个货物，起始位置的范围根据货物类型对齐
    lower_bounds = []
    upper_bounds = []
    for i in range(num_cargos):
        cargo_type = cargo_types[i]
        lower_bounds.append([0] * max_positions)
        upper_bounds.append([1] * max_positions)

    bounds = (np.array(lower_bounds), np.array(upper_bounds))

    # 初始化PSO优化器
    optimizer = ps.single.GlobalBestPSO(n_particles=swarmsize, dimensions=dimension, options=options, bounds=bounds)

    # 运行PSO优化
    best_cost, best_pos = optimizer.optimize(fitness_function, iters=maxiter)

    # 将最佳位置映射为离散装载方案
    best_start_positions = []
    penalty = 0
    cg_change = 0.0
    occupied = np.zeros(num_positions, dtype=int)

    for i in range(num_cargos):
        cargo_type = cargo_types[i]
        pos_continuous = best_pos[i * max_positions:(i + 1) * max_positions]

        # 根据粒子位置值选择最佳货位
        start_pos = np.argmax(pos_continuous)

        # 检查边界
        if start_pos < 0 or start_pos + cargo_type > num_positions:
            penalty += 1000
            best_start_positions.append(start_pos)
            continue

        # 检查对齐
        if cargo_type == 2 and start_pos % 2 != 0:
            penalty += 1000
        if cargo_type == 4 and start_pos % 4 != 0:
            penalty += 1000

        # 检查重叠
        if np.any(occupied[start_pos:start_pos + cargo_type]):
            penalty += 1000
        else:
            occupied[start_pos:start_pos + cargo_type] = 1

        best_start_positions.append(start_pos)

        # 计算重心变化量
        if cargo_type == 1:
            cg_change += abs(weights[i] * cg_impact[start_pos])
        elif cargo_type == 2:
            cg_change += abs(weights[i] * cg_impact_2u[start_pos // 2])
        elif cargo_type == 4:
            cg_change += abs(weights[i] * cg_impact_4u[start_pos // 4])

    total_cg_change = cg_change + penalty

    # 构建装载方案矩阵
    best_xij = np.zeros((num_cargos, num_positions), dtype=int)
    for i, start_pos in enumerate(best_start_positions):
        cargo_type = cargo_types[i]
        for k in range(cargo_type):
            pos = start_pos + k
            if pos < num_positions:
                best_xij[i, pos] = 1

    # 检查是否有严重惩罚，判断是否找到可行解
    if total_cg_change >= -999999:
        return best_xij, total_cg_change
    else:
        return [], -1000000


# 示例输入
weights = [500, 800, 1200, 300, 700, 1000, 600, 900]  # 每个货物的质量
cargo_names = ['LD3', 'LD3', 'PLA', 'LD3', 'P6P', 'PLA', 'LD3', 'BULK']  # 货物名称
cargo_types_dict = {"LD3": 1, "PLA": 2, "P6P": 4, "BULK": 1}  # 货物占位关系
positions = list(range(44))  # 44个货位编号
cg_impact = [i * 0.1 for i in range(44)]  # 每kg货物对重心index的影响系数 (单个位置)
cg_impact_2u = [i * 0.08 for i in range(22)]  # 两个位置组合的影响系数
cg_impact_4u = [i * 0.05 for i in range(11)]  # 四个位置组合的影响系数
max_positions = 44  # 总货位数量

# PSO算法配置选项
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

# 调用粒子群优化进行货物装载优化
best_solution, best_cg_change = cargo_load_planning_pso_v2(weights, cargo_names, cargo_types_dict, positions, 
                                                           cg_impact, cg_impact_2u, cg_impact_4u, max_positions,
                                                           options=options, swarmsize=100, maxiter=100)

# 输出结果
if best_solution.size > 0:
    print("最优装载方案矩阵：")
    print(best_solution)
    print(f"最优装载方案的重心变化量：{best_cg_change:.2f}")
else:
    print("优化失败，未找到可行解。")

## Example 11: PSO-normal


In [ ]:
import numpy as np
import pyswarms as ps


def cargo_load_planning_pso(weights, cargo_names, cargo_types_dict, positions, cg_impact, cg_impact_2u, cg_impact_4u,
                            max_positions, options=None, swarmsize=100, maxiter=100):
    """
    使用粒子群优化方法计算货物装载方案，最小化重心的变化量。

    参数:
        weights (list): 每个货物的质量列表。
        cargo_names (list): 每个货物的名称。
        cargo_types_dict (dict): 货物名称和占用的货位数量。
        positions (list): 可用的货位编号。
        cg_impact (list): 每个位置每kg货物对重心index的影响系数。
        cg_impact_2u (list): 两个位置组合的重心影响系数。
        cg_impact_4u (list): 四个位置组合的重心影响系数。
        max_positions (int): 总货位的数量。
        options (dict, optional): PSO算法的配置选项。
        swarmsize (int, optional): 粒子群大小。
        maxiter (int, optional): 最大迭代次数。

    返回:
        best_solution (np.array): 最优装载方案矩阵。
        best_cg_change (float): 最优方案的重心变化量。
    """
    # 将货物类型映射为对应的占用单位数
    cargo_types = [cargo_types_dict[name] for name in cargo_names]

    num_cargos = len(weights)  # 货物数量
    num_positions = len(positions)  # 可用货位数量
    dimension = num_cargos  # 每个粒子的维度对应每个货物的起始位置

    # 如果未提供options，使用默认配置
    if options is None:
        options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

    # 定义适应度评估函数
    def fitness_function(x):
        """
        计算每个粒子的适应度值。

        参数:
            x (numpy.ndarray): 粒子的位置数组，形状为 (n_particles, dimension)。

        返回:
            numpy.ndarray: 每个粒子的适应度值。
        """
        fitness = np.zeros(x.shape[0])

        for idx, particle in enumerate(x):
            # 将连续位置映射为离散起始位置，根据货物类型对齐
            start_positions = []
            penalty = 0
            cg_change = 0.0
            occupied = np.zeros(num_positions, dtype=int)

            for i in range(num_cargos):
                cargo_type = cargo_types[i]
                pos_continuous = particle[i]

                # 根据货物类型对齐
                if cargo_type == 1:
                    start_pos = int(np.round(pos_continuous)) % num_positions
                elif cargo_type == 2:
                    start_pos = (int(np.round(pos_continuous)) // 2) * 2
                    if start_pos >= num_positions - 1:
                        start_pos = num_positions - 2
                elif cargo_type == 4:
                    start_pos = (int(np.round(pos_continuous)) // 4) * 4
                    if start_pos >= num_positions - 3:
                        start_pos = num_positions - 4
                else:
                    start_pos = 0  # 默认位置

                # 检查边界
                if start_pos < 0 or start_pos + cargo_type > num_positions:
                    penalty += 1000
                    continue

                # 检查对齐
                if cargo_type == 2 and start_pos % 2 != 0:
                    penalty += 1000
                if cargo_type == 4 and start_pos % 4 != 0:
                    penalty += 1000

                # 检查重叠
                if np.any(occupied[start_pos:start_pos + cargo_type]):
                    penalty += 1000
                else:
                    occupied[start_pos:start_pos + cargo_type] = 1

                start_positions.append(start_pos)

                # 计算重心变化量
                if cargo_type == 1:
                    cg_change += weights[i] * cg_impact[start_pos]
                elif cargo_type == 2:
                    cg_change += weights[i] * cg_impact_2u[start_pos // 2]
                elif cargo_type == 4:
                    cg_change += weights[i] * cg_impact_4u[start_pos // 4]

            fitness[idx] = cg_change + penalty

        return fitness

    # 设置PSO的边界
    # 对于每个货物，起始位置的范围根据货物类型对齐
    lower_bounds = []
    upper_bounds = []
    for i in range(num_cargos):
        cargo_type = cargo_types[i]
        if cargo_type == 1:
            lower_bounds.append(0)
            upper_bounds.append(num_positions - 1)
        elif cargo_type == 2:
            lower_bounds.append(0)
            upper_bounds.append(num_positions - 2)
        elif cargo_type == 4:
            lower_bounds.append(0)
            upper_bounds.append(num_positions - 4)
        else:
            lower_bounds.append(0)
            upper_bounds.append(num_positions - 1)

    bounds = (np.array(lower_bounds), np.array(upper_bounds))

    # 初始化PSO优化器
    optimizer = ps.single.GlobalBestPSO(n_particles=swarmsize, dimensions=dimension, options=options, bounds=bounds)

    # 运行PSO优化
    best_cost, best_pos = optimizer.optimize(fitness_function, iters=maxiter)

    # 将最佳位置映射为离散装载方案
    best_start_positions = []
    penalty = 0
    cg_change = 0.0
    occupied = np.zeros(num_positions, dtype=int)

    for i in range(num_cargos):
        cargo_type = cargo_types[i]
        pos_continuous = best_pos[i]

        # 根据货物类型对齐
        if cargo_type == 1:
            start_pos = int(np.round(pos_continuous)) % num_positions
        elif cargo_type == 2:
            start_pos = (int(np.round(pos_continuous)) // 2) * 2
            if start_pos >= num_positions - 1:
                start_pos = num_positions - 2
        elif cargo_type == 4:
            start_pos = (int(np.round(pos_continuous)) // 4) * 4
            if start_pos >= num_positions - 3:
                start_pos = num_positions - 4
        else:
            start_pos = 0  # 默认位置

        # 检查边界
        if start_pos < 0 or start_pos + cargo_type > num_positions:
            penalty += 1000
            best_start_positions.append(start_pos)
            continue

        # 检查对齐
        if cargo_type == 2 and start_pos % 2 != 0:
            penalty += 1000
        if cargo_type == 4 and start_pos % 4 != 0:
            penalty += 1000

        # 检查重叠
        if np.any(occupied[start_pos:start_pos + cargo_type]):
            penalty += 1000
        else:
            occupied[start_pos:start_pos + cargo_type] = 1

        best_start_positions.append(start_pos)

        # 计算重心变化量
        if cargo_type == 1:
            cg_change += abs(weights[i] * cg_impact[start_pos])
        elif cargo_type == 2:
            cg_change += abs(weights[i] * cg_impact_2u[start_pos // 2])
        elif cargo_type == 4:
            cg_change += abs(weights[i] * cg_impact_4u[start_pos // 4])

    total_cg_change = cg_change + penalty

    # 构建装载方案矩阵
    best_xij = np.zeros((num_cargos, num_positions), dtype=int)
    for i, start_pos in enumerate(best_start_positions):
        cargo_type = cargo_types[i]
        for k in range(cargo_type):
            pos = start_pos + k
            if pos < num_positions:
                best_xij[i, pos] = 1

    # 检查是否有严重惩罚，判断是否找到可行解
    if total_cg_change >= -999999:
        return best_xij, total_cg_change
    else:
        return [], -1000000


# 示例输入和调用
def main():
    weights = [500, 800, 1200, 300, 700, 1000, 600, 900]  # 每个货物的质量
    cargo_names = ['LD3', 'LD3', 'PLA', 'LD3', 'P6P', 'PLA', 'LD3', 'BULK']  # 货物名称
    cargo_types_dict = {"LD3": 1, "PLA": 2, "P6P": 4, "BULK": 1}  # 货物占位关系
    positions = list(range(44))  # 44个货位编号
    cg_impact = [i * 0.1 for i in range(44)]  # 每kg货物对重心index的影响系数 (单个位置)
    cg_impact_2u = [i * 0.08 for i in range(22)]  # 两个位置组合的影响系数
    cg_impact_4u = [i * 0.05 for i in range(11)]  # 四个位置组合的影响系数
    max_positions = 44  # 总货位数量

    # 设置PSO参数（可选）
    options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

    solution, cg_change = cargo_load_planning_pso(
        weights, cargo_names, cargo_types_dict, positions,
        cg_impact, cg_impact_2u, cg_impact_4u, max_positions,
        options=options, swarmsize=200, maxiter=200
    )

    if solution is not None and len(solution) > 0:
        print("成功找到最优装载方案！")
        print("装载方案矩阵:")
        print(solution)
        print(f"重心的变化量: {cg_change:.2f}")

        # 输出实际分布
        for i in range(len(weights)):
            assigned_positions = []
            for j in range(len(positions)):
                if solution[i, j] > 0.5:  # 判断位置是否被分配
                    assigned_positions.append(j)
            print(f"货物 {cargo_names[i]} (占 {cargo_types_dict[cargo_names[i]]} 单位): 放置位置 -> {assigned_positions}")
    else:
        print("未找到可行的装载方案。")


if __name__ == "__main__":
    main()



## Example 12: RCH

In [ ]:
import numpy as np


def cargo_load_planning_heuristic(weights, cargo_names, cargo_types_dict, positions,
                                  cg_impact, cg_impact_2u, cg_impact_4u, max_positions,
                                  max_iterations=1000):
    """
    使用两阶段启发式算法计算货物装载方案，最小化重心的变化量。

    参数:
        weights (list): 每个货物的质量列表。
        cargo_names (list): 每个货物的名称。
        cargo_types_dict (dict): 货物名称和占用的货位数量。
        positions (list): 可用的货位编号。
        cg_impact (list): 每个位置每kg货物对重心index的影响系数。
        cg_impact_2u (list): 两个位置组合的重心影响系数。
        cg_impact_4u (list): 四个位置组合的重心影响系数。
        max_positions (int): 总货位的数量。
        max_iterations (int): 优化阶段的最大迭代次数。

    返回:
        solution (np.array): 最优装载方案矩阵。
        total_cg_change (float): 最优方案的重心变化量。
    """
    # 将货物类型映射为对应的占用单位数
    cargo_types = [cargo_types_dict[name] for name in cargo_names]

    num_cargos = len(weights)  # 货物数量
    num_positions = len(positions)  # 可用货位数量

    # 初始化装载方案矩阵
    solution = np.zeros((num_cargos, num_positions), dtype=int)

    # 标记已占用的位置
    occupied = np.zeros(num_positions, dtype=int)

    # 按照货物类型（占用单位数降序）和重量降序排序货物
    sorted_indices = sorted(range(num_cargos), key=lambda i: (-cargo_types[i], -weights[i]))

    # 阶段1：初始装载方案生成
    for i in sorted_indices:
        cargo_type = cargo_types[i]
        feasible_positions = []

        # 根据货物类型确定可行的起始位置
        if cargo_type == 1:
            possible_starts = range(0, num_positions)
        elif cargo_type == 2:
            possible_starts = range(0, num_positions - 1, 2)
        elif cargo_type == 4:
            possible_starts = range(0, num_positions - 3, 4)
        else:
            possible_starts = []

        for start in possible_starts:
            # 检查是否超出边界
            if start + cargo_type > num_positions:
                continue
            # 检查是否与已占用位置重叠
            if np.any(occupied[start:start + cargo_type]):
                continue
            # 计算重心变化量
            if cargo_type == 1:
                cg = abs(weights[i] * cg_impact[start])
            elif cargo_type == 2:
                cg = abs(weights[i] * cg_impact_2u[start // 2])
            elif cargo_type == 4:
                cg = abs(weights[i] * cg_impact_4u[start // 4])
            feasible_positions.append((start, cg))

        # 如果有可行位置，选择使重心变化最小的位置
        if feasible_positions:
            best_start, best_cg = min(feasible_positions, key=lambda x: x[1])
            solution[i, best_start:best_start + cargo_type] = 1
            occupied[best_start:best_start + cargo_type] = 1
        else:
            # 如果没有可行位置，则尝试分配到任何未占用的位置（可能违反约束）
            for start in range(0, num_positions - cargo_type + 1):
                if np.all(occupied[start:start + cargo_type] == 0):
                    solution[i, start:start + cargo_type] = 1
                    occupied[start:start + cargo_type] = 1
                    break

    # 计算初始重心变化量
    total_cg_change = 0.0
    for i in range(num_cargos):
        cargo_type = cargo_types[i]
        assigned_positions = np.where(solution[i] == 1)[0]
        if len(assigned_positions) == 0:
            continue
        if cargo_type == 1:
            total_cg_change += abs(weights[i] * cg_impact[assigned_positions[0]])
        elif cargo_type == 2:
            total_cg_change += abs(weights[i] * cg_impact_2u[assigned_positions[0] // 2])
        elif cargo_type == 4:
            total_cg_change += abs(weights[i] * cg_impact_4u[assigned_positions[0] // 4])

    # 阶段2：装载方案优化
    for _ in range(max_iterations):
        improved = False
        for i in range(num_cargos):
            cargo_type = cargo_types[i]
            current_positions = np.where(solution[i] == 1)[0]
            if len(current_positions) == 0:
                continue
            current_start = current_positions[0]

            # 根据货物类型确定可行的起始位置
            if cargo_type == 1:
                possible_starts = range(0, num_positions)
            elif cargo_type == 2:
                possible_starts = range(0, num_positions - 1, 2)
            elif cargo_type == 4:
                possible_starts = range(0, num_positions - 3, 4)
            else:
                possible_starts = []

            best_start = current_start
            best_cg = total_cg_change

            for start in possible_starts:
                if start == current_start:
                    continue
                # 检查是否超出边界
                if start + cargo_type > num_positions:
                    continue
                # 检查是否与已占用位置重叠
                if np.any(occupied[start:start + cargo_type]):
                    continue
                # 计算重心变化量
                if cargo_type == 1:
                    new_cg = abs(weights[i] * cg_impact[start])
                elif cargo_type == 2:
                    new_cg = abs(weights[i] * cg_impact_2u[start // 2])
                elif cargo_type == 4:
                    new_cg = abs(weights[i] * cg_impact_4u[start // 4])
                else:
                    new_cg = 0

                # 计算新的总重心变化量
                temp_cg_change = total_cg_change - (
                    weights[i] * cg_impact[current_start] if cargo_type == 1 else
                    weights[i] * cg_impact_2u[current_start // 2] if cargo_type == 2 else
                    weights[i] * cg_impact_4u[current_start // 4] if cargo_type == 4 else 0
                ) + new_cg

                # 如果新的重心变化量更小，进行更新
                if temp_cg_change < best_cg:
                    best_cg = temp_cg_change
                    best_start = start

            # 如果找到了更好的位置，进行更新
            if best_start != current_start:
                # 释放当前占用的位置
                occupied[current_start:current_start + cargo_type] = 0
                solution[i, current_start:current_start + cargo_type] = 0

                # 分配到新的位置
                solution[i, best_start:best_start + cargo_type] = 1
                occupied[best_start:best_start + cargo_type] = 1

                # 更新总重心变化量
                total_cg_change = best_cg
                improved = True

        if not improved:
            break  # 如果在一个完整的迭代中没有改进，结束优化

    return solution, total_cg_change


# 示例输入和调用
def main():
    weights = [500, 800, 1200, 300, 700, 1000, 600, 900]  # 每个货物的质量
    cargo_names = ['LD3', 'LD3', 'PLA', 'LD3', 'P6P', 'PLA', 'LD3', 'BULK']  # 货物名称
    cargo_types_dict = {"LD3": 1, "PLA": 2, "P6P": 4, "BULK": 1}  # 货物占位关系
    positions = list(range(44))  # 44个货位编号
    cg_impact = [i * 0.1 for i in range(44)]  # 每kg货物对重心index的影响系数 (单个位置)
    cg_impact_2u = [i * 0.08 for i in range(22)]  # 两个位置组合的影响系数
    cg_impact_4u = [i * 0.05 for i in range(11)]  # 四个位置组合的影响系数
    max_positions = 44  # 总货位数量

    solution, cg_change = cargo_load_planning_heuristic(
        weights, cargo_names, cargo_types_dict, positions,
        cg_impact, cg_impact_2u, cg_impact_4u, max_positions,
        max_iterations=1000
    )

    if solution is not None and solution.size > 0:
        print("成功找到最优装载方案！")
        print("装载方案矩阵:")
        print(solution)
        print(f"重心的变化量: {cg_change:.2f}")

        # 输出实际分布
        for i in range(len(weights)):
            assigned_positions = []
            for j in range(len(positions)):
                if solution[i, j] > 0.5:  # 判断位置是否被分配
                    assigned_positions.append(j)
            print(f"货物 {cargo_names[i]} (占 {cargo_types_dict[cargo_names[i]]} 单位): 放置位置 -> {assigned_positions}")
    else:
        print("未找到可行的装载方案。")


if __name__ == "__main__":
    main()
